In [17]:
import pandas as pd
import numpy as np
from supervised.automl import AutoML
from dateutil.relativedelta import relativedelta

In [18]:
df_canada = pd.read_csv('canada.csv')
df_canada.head()

,date,QUALITY_FLAG,cid,industry_raw,E_TTM_period_date,E_TTM_ammor_intangibles,E_TTM_asset_writedown,E_TTM_assets_gro_five,E_TTM_capex,E_TTM_cash_acquisitions,...,E_G_cost_debt,E_G_cash_ratio,E_G_ebitda_cov,E_G_ret_on_asset,E_G_ret_on_inv_cap,E_G_net_to_cash,E_G_perm_assets_ratio,return_1q,target_net_income,target_cash_operations
0,2002-01-03,True,SP-065996,NaN,2001-10-31,0.000,0.0,0.0,-12.738000,-3.336,...,-0.036902,2.309257,-165.453488,0.130018,0.101871,-0.068216,0.414230,NaN,-5.548000,-5.809000
1,2002-01-08,True,SP-002396,NaN,2001-09-30,3.078,0.0,0.0,-20.889000,-68.220,...,-0.103778,0.000000,-2.685925,0.071119,0.067430,-0.004881,0.595752,NaN,6.114000,-11.388000
2,2002-01-08,True,SP-006704,NaN,2001-09-30,0.000,0.0,0.0,-17.971623,0.000,...,-0.036358,2.451612,2.262460,-0.069781,-0.039238,-0.045993,0.775432,NaN,8.957867,-1.346965
3,2002-01-08,True,SP-008644,NaN,2001-09-30,0.000,0.0,0.0,-34.700000,0.000,...,-0.117333,0.878049,-4.852273,-0.169833,-0.155712,-0.316372,0.773996,NaN,71.200000,-9.400000
4,2002-01-08,True,SP-013994,NaN,2001-09-30,0.000,0.0,0.0,-1403.000000,-133.000,...,-0.078051,0.000000,-14.569697,0.109798,0.078497,-0.157934,0.921832,NaN,-165.000000,-419.000000


In [19]:
df_canada.tail()

,date,QUALITY_FLAG,cid,industry_raw,E_TTM_period_date,E_TTM_ammor_intangibles,E_TTM_asset_writedown,E_TTM_assets_gro_five,E_TTM_capex,E_TTM_cash_acquisitions,...,E_G_cost_debt,E_G_cash_ratio,E_G_ebitda_cov,E_G_ret_on_asset,E_G_ret_on_inv_cap,E_G_net_to_cash,E_G_perm_assets_ratio,return_1q,target_net_income,target_cash_operations
47360,2024-12-12,True,SP-035955,"Oil, Gas & Consumable Fuels",2024-10-31,0.030,0.0,47.4070,0.000,0.000,...,-1.395745,0.636431,0.000000,0.016859,0.021506,0.381318,0.194369,NaN,NaN,NaN
47361,2024-12-12,True,SP-063476,Software,2024-10-31,17.107,0.0,7.1326,-1.977,-43.277,...,-0.054563,1.298073,-189.097521,0.098301,0.118097,-0.060880,0.540278,NaN,NaN,NaN
47362,2024-12-12,True,SP-064686,Specialty Retail,2024-10-31,0.000,0.0,9.6318,-20.609,0.000,...,0.000000,0.000000,NaN,0.068533,-0.002755,0.046886,0.831640,NaN,NaN,NaN
47363,2024-12-12,True,SP-122154,Metals & Mining,2024-10-31,0.435,0.0,10.5561,-8.181,0.000,...,-0.058242,0.760917,-32.123982,0.197987,0.247023,-0.088436,0.377709,NaN,NaN,NaN
47364,2024-12-13,True,SP-064624,Energy Equipment & Services,2024-09-30,0.000,0.0,0.0000,0.000,0.000,...,0.000000,NaN,NaN,NaN,0.112821,NaN,NaN,NaN,NaN,NaN


In [37]:


df_canada["EPS"] = df_canada["target_net_income"] / df_canada["E_TTM_shares_out"]

In [38]:
df_model = df_canada.copy()

df_model['date'] = pd.to_datetime(df_model['date'], errors='coerce')
df_model.sort_values(by=['cid', 'date'], inplace=True)

In [22]:
ratio_keywords = ['_on_assets_ratio', '_on_rev_ratio', '_on_tot_cap_ratio']

for rk in ratio_keywords:
    # lister toutes les colonnes contenant rk
    ratio_cols = [col for col in df_model.columns if rk in col]
    print(f"{rk} : {len(ratio_cols)} colonnes → {ratio_cols}")
    
for rk in ratio_keywords:
    print(rk, sum(rk in col for col in df_model.columns))

_on_assets_ratio : 64 colonnes → ['E_G_ammor_intangibles_on_assets_ratio', 'E_G_asset_writedown_on_assets_ratio', 'E_G_assets_gro_five_on_assets_ratio', 'E_G_capex_on_assets_ratio', 'E_G_cash_acquisitions_on_assets_ratio', 'E_G_cash_equi_on_assets_ratio', 'E_G_cash_financing_on_assets_ratio', 'E_G_cash_investing_on_assets_ratio', 'E_G_cash_operations_on_assets_ratio', 'E_G_cogs_on_assets_ratio', 'E_G_common_equity_on_assets_ratio', 'E_G_compre_income_on_assets_ratio', 'E_G_core_banks_on_assets_ratio', 'E_G_cost_borrowing_on_assets_ratio', 'E_G_current_assets_on_assets_ratio', 'E_G_current_liabilities_on_assets_ratio', 'E_G_def_tax_assets_on_assets_ratio', 'E_G_def_tax_liability_on_assets_ratio', 'E_G_depre_amor_on_assets_ratio', 'E_G_divestitures_on_assets_ratio', 'E_G_ebit_on_assets_ratio', 'E_G_ebitda_on_assets_ratio', 'E_G_ebt_x_unusual_on_assets_ratio', 'E_G_effective_tax_rate_on_assets_ratio', 'E_G_gross_profit_on_assets_ratio', 'E_G_gross_property_on_assets_ratio', 'E_G_int_div_i

In [28]:
# Retirer les lignes où Quality_Flag est False
df_model = df_model[df_model['QUALITY_FLAG'] == True]

# (FACULTATIF) Exclure les banques
# df_model = df_model[df_model['industry'] != 'Banks']

In [24]:
# 6) Optionnel : On peut réduire le DataFrame aux seules colonnes qu’on veut.
#    Exemple : On veut toutes les colonnes "E_TTM_..." + "target" + "date" + "sid"
#    On exclude juste ce qui est inutile explicitement.
columns_to_exclude = [
    'target_cash_operation',
    'return_1Q',
    'target_net_income',  # car on vient de le renommer en "target"
    'QUALITY_FLAG',
    # etc. toute colonne que tu ne veux pas passer à l’entraînement
]

columns_to_keep = [col for col in df_model.columns if col not in columns_to_exclude]
df_model = df_model[columns_to_keep]

# (Vérifier qu’on a bien "target", "date" et "sid" dedans)

In [25]:
df_model.head()

,date,cid,industry_raw,E_TTM_period_date,E_TTM_ammor_intangibles,E_TTM_asset_writedown,E_TTM_assets_gro_five,E_TTM_capex,E_TTM_cash_acquisitions,E_TTM_cash_equi,...,E_G_cost_debt,E_G_cash_ratio,E_G_ebitda_cov,E_G_ret_on_asset,E_G_ret_on_inv_cap,E_G_net_to_cash,E_G_perm_assets_ratio,return_1q,target_cash_operations,EPS
110,2002-02-11,MISSING,NaN,2001-12-31,4060.000000,0.000,116.6448,-471.500000,129.400000,215.300000,...,0.000000,0.306564,NaN,-6.411262,-0.611325,-6.406740,0.726575,NaN,30.345,0.068111
126,2002-02-14,MISSING,NaN,2001-12-31,25.632000,-0.948,116.6448,-4.323000,-2.406000,14.602000,...,0.000000,0.362873,NaN,-1.238392,-1.318019,-1.029941,0.240561,NaN,30.345,0.481380
307,2002-03-12,MISSING,NaN,2001-12-31,25.632000,-0.948,116.6448,-4.323000,-2.406000,1.123000,...,NaN,0.362258,NaN,0.052164,0.050056,0.052724,0.988220,NaN,30.345,0.340005
360,2002-04-04,MISSING,NaN,2001-12-31,199.720473,-0.948,64.2833,-98.911306,-452.928646,208.936372,...,-0.056982,0.224200,-10.583740,0.047535,0.022288,-0.051448,0.712545,NaN,30.345,0.028921
365,2002-04-05,MISSING,NaN,2001-12-31,4.108000,-0.948,64.2833,-26.251000,-33.390000,1.935000,...,-0.161252,0.049053,-2.743061,0.060979,0.061096,-0.038678,0.941717,NaN,30.345,0.321925


In [29]:
# 5) Définir la liste des colonnes "ratios" qu’on veut conserver
# ---------------------------
ratio_keywords = [
    '_on_assets_ratio',
    '_on_rev_ratio',
    '_on_tot_cap_ratio'
]

# On va chercher toutes les colonnes qui contiennent l’un de ces substrings
ratio_cols = [
    col for col in df_model.columns
    if any(rk in col for rk in ratio_keywords)
]

# ---------------------------
# 6) Définir les colonnes minimales qu’on veut absolument
# ---------------------------
mandatory_cols = ['date', 'cid', 'target_net_income']

# ---------------------------
# 7) Construire la liste finale des colonnes à garder
# ---------------------------
columns_to_keep = mandatory_cols + ratio_cols

# ---------------------------
# 8) Créer le df_model_final avec seulement ces colonnes
# ---------------------------
df_model_final = df_model[columns_to_keep].copy()

# ---------------------------
# 9) (Optionnel) trier par (cid, date)
# ---------------------------
df_model_final.sort_values(by=['cid', 'date'], inplace=True)


In [30]:
df_model_final

,date,cid,target_net_income,E_G_ammor_intangibles_on_assets_ratio,E_G_asset_writedown_on_assets_ratio,E_G_assets_gro_five_on_assets_ratio,E_G_capex_on_assets_ratio,E_G_cash_acquisitions_on_assets_ratio,E_G_cash_equi_on_assets_ratio,E_G_cash_financing_on_assets_ratio,...,E_G_total_assets_on_tot_cap_ratio,E_G_total_capital_on_tot_cap_ratio,E_G_total_debt_on_tot_cap_ratio,E_G_total_div_on_tot_cap_ratio,E_G_total_expenses_on_tot_cap_ratio,E_G_total_intangibles_on_tot_cap_ratio,E_G_total_liabilities_on_tot_cap_ratio,E_G_total_revenues_on_tot_cap_ratio,E_G_unusual_items_on_tot_cap_ratio,E_G_working_cap_on_tot_cap_ratio
110,2002-02-11,MISSING,11.556,0.452181,0.000000,0.012991,-0.052513,0.014412,0.023979,0.011884,...,1.165218,1.0,0.000740,0.000000,0.889496,0.667385,0.165957,0.277188,-6.882254,0.227458
126,2002-02-14,MISSING,11.556,0.294154,-0.010879,1.338621,-0.049611,-0.027611,0.167573,0.033923,...,1.972073,1.0,0.018286,0.000000,3.142851,0.341918,0.990359,1.833748,-1.185375,0.586973
307,2002-03-12,MISSING,11.556,0.093195,-0.003447,0.424109,-0.015718,-0.008748,0.004083,0.034563,...,1.011400,1.0,0.000000,-0.097703,0.001000,0.055557,0.011400,0.053759,-0.192609,0.000515
360,2002-04-04,MISSING,11.556,0.030985,-0.000147,0.009973,-0.015345,-0.070268,0.032415,-0.015248,...,1.358680,1.0,0.300623,-0.025311,0.378268,0.825734,0.659304,0.496560,-0.018392,0.194123
365,2002-04-05,MISSING,11.556,0.008603,-0.001985,0.134617,-0.054973,-0.069923,0.004052,0.060828,...,1.106583,1.0,0.255268,-0.057910,0.327643,0.181478,0.361851,0.398861,-0.202194,-0.026916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46059,2024-03-07,SP-329579,-2.280,0.000298,0.000000,0.186328,-0.194598,0.000000,0.221897,0.232036,...,1.203700,1.0,0.063159,0.000000,0.658416,0.011760,0.266859,0.465194,0.035120,0.193336
46510,2024-05-15,SP-329579,0.071,0.000410,-0.043386,0.155120,-0.336551,0.000000,0.154666,0.297958,...,1.118280,1.0,0.041826,0.000000,0.566620,0.011920,0.160106,0.436938,-0.053453,0.240717
46855,2024-08-08,SP-329579,-25.028,0.000000,0.000000,0.119845,-0.371029,0.000000,0.258649,0.465630,...,1.094315,1.0,0.032399,0.000000,0.502595,0.008939,0.126713,0.352637,0.001327,0.389704
47295,2024-11-13,SP-329579,NaN,0.000000,-0.006185,0.106844,-0.493308,0.000000,0.124292,0.494820,...,1.094493,1.0,0.044694,0.000000,0.519705,0.007851,0.139186,0.341355,-0.013015,0.257389


In [31]:
# 1) Compter le nombre total de NaN dans tout le DataFrame
total_nan = df_model_final.isna().sum().sum()
print(f"Nombre total de valeurs NaN dans df_model_final : {total_nan}")

# 2) Retirer les lignes qui contiennent AU MOINS un NaN
df_model_final.dropna(inplace=True)

# Vérifier à nouveau qu’il n’y a plus de NaN
total_nan_apres = df_model_final.isna().sum().sum()
print(f"Nombre total de valeurs NaN après suppression : {total_nan_apres}")

Nombre total de valeurs NaN dans df_model_final : 124606
Nombre total de valeurs NaN après suppression : 0


In [32]:
# 1) Date la plus récente du DataFrame
max_date = df_model_final['date'].max()

# 2) Date de coupure (5 ans avant)
cutoff_date = max_date - pd.DateOffset(years=5)

# 3) Filtrer pour ne garder que les 5 dernières années
df_test = df_model_final[df_model_final['date'] >= cutoff_date].copy()

print(df_test['date'].min(), df_test['date'].max())
print(df_test.shape)

2019-09-24 00:00:00 2024-09-19 00:00:00
(7730, 195)


In [33]:
df_test

,date,cid,target_net_income,E_G_ammor_intangibles_on_assets_ratio,E_G_asset_writedown_on_assets_ratio,E_G_assets_gro_five_on_assets_ratio,E_G_capex_on_assets_ratio,E_G_cash_acquisitions_on_assets_ratio,E_G_cash_equi_on_assets_ratio,E_G_cash_financing_on_assets_ratio,...,E_G_total_assets_on_tot_cap_ratio,E_G_total_capital_on_tot_cap_ratio,E_G_total_debt_on_tot_cap_ratio,E_G_total_div_on_tot_cap_ratio,E_G_total_expenses_on_tot_cap_ratio,E_G_total_intangibles_on_tot_cap_ratio,E_G_total_liabilities_on_tot_cap_ratio,E_G_total_revenues_on_tot_cap_ratio,E_G_unusual_items_on_tot_cap_ratio,E_G_working_cap_on_tot_cap_ratio
38227,2019-10-31,MISSING,408.247,0.000000,0.000000,0.002422,-0.049673,-0.195067,0.135870,0.126521,...,1.208425,1.0,0.633449,-0.020783,0.905598,0.033124,0.841874,1.057073,-0.004895,0.358843
38711,2020-02-13,MISSING,-580.985,0.002871,0.000000,0.004647,-0.063917,-0.003089,0.169958,0.002952,...,1.225347,1.0,0.673511,-0.020927,0.913769,0.031659,0.898858,0.963571,-0.002975,0.349019
38932,2020-03-06,MISSING,-580.985,0.001827,0.000000,0.002958,-0.040689,-0.001966,0.015012,-0.002633,...,1.038354,1.0,0.175109,-0.011289,0.026011,0.017078,0.213462,0.232473,-0.001605,-0.169798
39140,2020-04-30,MISSING,-18.670,0.003038,0.000000,0.004924,-0.069500,-0.003268,0.146787,0.034360,...,1.185758,1.0,0.718886,-0.026926,0.896837,0.029739,0.904644,0.905761,-0.003046,0.366548
39627,2020-07-30,MISSING,6.338,0.003005,0.000000,0.004689,-0.066998,-0.003233,0.153695,0.040398,...,1.191067,1.0,0.701716,-0.021798,0.856682,0.030110,0.892783,0.848820,-0.003027,0.354061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45117,2023-08-08,SP-329579,66.105,0.000000,-0.230972,0.135011,-0.271879,0.000000,0.094863,-0.083550,...,1.069493,1.0,0.104221,0.000000,0.653627,0.000069,0.173714,0.413209,-0.141642,0.113785
45471,2023-11-06,SP-329579,-40.530,0.000048,0.026752,0.000000,-0.220474,0.000000,0.140551,0.041274,...,1.060528,1.0,0.083820,0.000000,0.648063,0.012176,0.144349,0.405229,0.101841,0.172232
46059,2024-03-07,SP-329579,-2.280,0.000298,0.000000,0.186328,-0.194598,0.000000,0.221897,0.232036,...,1.203700,1.0,0.063159,0.000000,0.658416,0.011760,0.266859,0.465194,0.035120,0.193336
46510,2024-05-15,SP-329579,0.071,0.000410,-0.043386,0.155120,-0.336551,0.000000,0.154666,0.297958,...,1.118280,1.0,0.041826,0.000000,0.566620,0.011920,0.160106,0.436938,-0.053453,0.240717


In [34]:
def pipeline_rolling_windows(data, date_col, target_col, train_years, val_years, test_years, buffer_months=0):
    """
    Pipeline direct pour la rolling window avec AutoML et cross-validation personnalisée.
    Ajoute les périodes dans le DataFrame final pour validation.
    """
    # Conversion de la colonne date
    data[date_col] = pd.to_datetime(data[date_col])
    start_date = data[date_col].min()
    end_date = data[date_col].max()

    predictions_all = []  # Liste pour stocker toutes les prédictions

    while start_date + relativedelta(years=train_years + val_years + test_years) <= end_date:
        # Définir les périodes
        train_end = start_date + relativedelta(years=train_years) - pd.Timedelta(days=1)
        tampon_1_end = train_end + relativedelta(months=buffer_months)
        val_start = tampon_1_end + pd.Timedelta(days=1)
        val_end = val_start + relativedelta(years=val_years) - pd.Timedelta(days=1)
        tampon_2_end = val_end + relativedelta(months=buffer_months)
        test_start = tampon_2_end + pd.Timedelta(days=1)
        test_end = test_start + relativedelta(years=test_years) - pd.Timedelta(days=1)

        # Filtrer les données
        train_data = data.loc[(data[date_col] >= start_date) & (data[date_col] <= train_end)]
        val_data = data.loc[(data[date_col] >= val_start) & (data[date_col] <= val_end)]
        test_data = data.loc[(data[date_col] >= test_start) & (data[date_col] <= test_end)]

        if len(train_data) == 0 or len(val_data) == 0 or len(test_data) == 0:
            print(f"Fenêtre {start_date.year}-{test_end.year} : données insuffisantes, sautée.")
            start_date += relativedelta(years=1)
            continue

        # Configurer et entraîner AutoML
        print(f"Fenêtre {start_date.year}-{test_end.year} : entraînement de AutoML...")
        automl = AutoML(mode="Perform", algorithms=["Xgboost"])
        custom_cv = [(train_data.index, val_data.index)]
        automl.fit(
            train_data.drop(columns=[target_col, date_col, 'cid']),
            train_data[target_col], cv=custom_cv
        )

        # Prédire sur le test set
        test_preds = test_data[[date_col, target_col]].copy()
        test_preds["predicted"] = automl.predict(test_data.drop(columns=[target_col, date_col, 'cid']))
        test_preds["window"] = f"{start_date.year}-{test_end.year}"
        test_preds["cid"] = test_data["cid"].values

        # Ajouter les périodes pour validation
        test_preds["train_start"] = start_date
        test_preds["train_end"] = train_end
        test_preds["tampon_1"] = tampon_1_end
        test_preds["val_start"] = val_start
        test_preds["val_end"] = val_end
        test_preds["tampon_2"] = tampon_2_end
        test_preds["test_start"] = test_start
        test_preds["test_end"] = test_end

        # Sauvegarder les prédictions
        predictions_all.append(test_preds)

        # Avancer la fenêtre
        start_date += relativedelta(years=1)

    predictions_df = pd.concat(predictions_all, ignore_index=True)
    return predictions_df

In [35]:
predictions_df = pipeline_rolling_windows(
    data=df_test, 
    date_col="date", 
    target_col="target_net_income", 
    train_years=2, 
    val_years=1, 
    test_years=1, 
    buffer_months=1
)

Fenêtre 2019-2023 : entraînement de AutoML...
AutoML directory: AutoML_1
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Xgboost']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'ensemble']
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 1 model
1_Default_Xgboost rmse 392.713046 trained in 20.6 seconds (1-sample predict time 0.0352 seconds)
* Step not_so_random will try to check up to 4 models
2_Xgboost rmse 393.380887 trained in 10.32 seconds (1-sample predict time 0.0351 seconds)
3_Xgboost rmse 387.136486 trained in 8.54 seconds (1-sample predict time 0.0349 seconds)
4_Xgboost rmse 389.565548 trained in 11.01 seconds (1-sample predict time 0.035 seconds)
5_Xgboost rmse 404.221721 trained in 10.7 seconds (1-sample predict t

In [36]:
predictions_df.to_csv("df_can_test.csv", index=False)

In [39]:
# bucket les pred en déciles, pour chaque fenêtre, sum des buckets